In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.3.1 --progress-bar off
!pip install -qqq transformers==4.41.3 --progress-bar off
!pip install -qqq datasets==2.20.0 --progress-bar off
!pip install -qqq accelerate==0.31.0 --progress-bar off
!pip install -qqq bitsandbytes==0.43.1 --progress-bar off
!pip install -qqq peft==0.11.1 --progress-bar off
!pip install -qqq trl==0.9.4 --progress-bar off
!pip install -qqq colored==2.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
ERROR: Ignored the following yanked versions: 4.14.0, 4.25.0
ERROR: Could not find a version that satisfies the requirement transformers==4.41.3 (from versions: 0.1, 2.0.0, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.8.0, 2.9.0, 2.9.1, 2.10.0, 2.11.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.3.1, 3.4.0, 3.5.0, 3.5.1, 4.0.0rc1, 4.0.0, 4.0.1, 4.1.0, 4.1.1, 4.2.0, 4.2.1, 4.2.2, 4.3.0rc1, 4.3.0, 4.3.1, 4.3.2, 4.3.3, 4.4.0, 4.4.1, 4.4.2, 4.5.0, 4.5.1, 4.6.0, 4.6.1, 4.7.0, 4.8.0, 4.8.1, 4.8.2, 4.9.0, 4.9.1, 4.9.2, 4.10.0, 4.10.1, 4.10.2, 4.10.3, 4.11.0, 4.11.1, 4.11.2, 4.11.3, 4.12.0, 4.12

In [3]:
import random
from textwrap import dedent
from typing import Dict, List

import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from colored import Back, Fore, Style
from datasets import Dataset, load_dataset
from matplotlib.ticker import PercentFormatter
from peft import (
    LoraConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer



SEED = 42
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


seed_everything(SEED)
PAD_TOKEN = "<|pad|>"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
NEW_MODEL = "Llama-3-8B-Instruct-Finance-RAG"

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Embedding(128264, 4096)

In [4]:
model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
 

In [5]:
tokenizer.bos_token, tokenizer.bos_token_id

('<|begin_of_text|>', 128000)

In [6]:
tokenizer.eos_token, tokenizer.eos_token_id

('<|eot_id|>', 128009)

In [7]:
tokenizer.pad_token, tokenizer.pad_token_id

('<|pad|>', 128256)

In [8]:
tokenizer.convert_tokens_to_ids(PAD_TOKEN)

128256

In [9]:
dataset = load_dataset("virattt/financial-qa-10K")

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context', 'ticker', 'filing'],
        num_rows: 7000
    })
})

In [11]:
dataset["train"][:10]

{'question': ['What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
  'What are some of the recent applications of GPU-powered deep learning as mentioned by NVIDIA?',
  'What significant invention did NVIDIA create in 1999?',
  "How does NVIDIA's platform strategy contribute to the markets it serves?",
  "What does NVIDIA's CUDA programming model enable?",
  "What industries use NVIDIA's GPUs and software for automation?",
  'Why did NVIDIA and SoftBank terminate their Share Purchase Agreement for acquiring Arm Limited?',
  'What amount did NVIDIA record as an acquisition termination cost in fiscal year 2023?',
  'What does the NVIDIA computing platform focus on accelerating?',
  'What are the key components of the NVIDIA computing platform?'],
 'answer': ['NVIDIA initially focused on PC graphics.',
  'Recent applications of GPU-powered deep learning include recommendation systems, large language models, and generative AI.',
  'NVIDIA in

In [12]:
rows = []
for item in dataset["train"]:
    rows.append(
        {
            "question": item["question"],
            "context": item["context"],
            "answer": item["answer"],
        }
    )
df = pd.DataFrame(rows)

In [13]:
df.head(5)

,question,context,answer
0,What area did NVIDIA initially focus on before...,"Since our original focus on PC graphics, we ha...",NVIDIA initially focused on PC graphics.
1,What are some of the recent applications of GP...,Some of the most recent applications of GPU-po...,Recent applications of GPU-powered deep learni...
2,What significant invention did NVIDIA create i...,Our invention of the GPU in 1999 defined moder...,NVIDIA invented the GPU in 1999.
3,How does NVIDIA's platform strategy contribute...,"NVIDIA has a platform strategy, bringing toget...",NVIDIA's platform strategy brings together har...
4,What does NVIDIA's CUDA programming model enable?,With our introduction of the CUDA programming ...,NVIDIA's CUDA programming model opened the par...


In [14]:
df.isnull().value_counts()

question  context  answer
False     False    False     7000
Name: count, dtype: int64

In [15]:
def format_example(row: dict):
    prompt = dedent(
        f"""
    {row["question"]}

    Information:

    ```
    {row["context"]}
    ```
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Use only the information to answer the question",
        },
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": row["answer"]},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)

In [16]:
df["text"] = df.apply(format_example, axis=1)

In [19]:
df.head(5)['text']

0    <|begin_of_text|><|start_header_id|>system<|en...
1    <|begin_of_text|><|start_header_id|>system<|en...
2    <|begin_of_text|><|start_header_id|>system<|en...
3    <|begin_of_text|><|start_header_id|>system<|en...
4    <|begin_of_text|><|start_header_id|>system<|en...
Name: text, dtype: object

In [20]:
def count_tokens(row: Dict) -> int:
    return len(
        tokenizer(
            row["text"],
            add_special_tokens=True,
            return_attention_mask=False,
        )["input_ids"]
    )

In [21]:
df["token_count"] = df.apply(count_tokens, axis=1)

In [22]:
df.head(5)

,question,context,answer,text,token_count
0,What area did NVIDIA initially focus on before...,"Since our original focus on PC graphics, we ha...",NVIDIA initially focused on PC graphics.,<|begin_of_text|><|start_header_id|>system<|en...,76
1,What are some of the recent applications of GP...,Some of the most recent applications of GPU-po...,Recent applications of GPU-powered deep learni...,<|begin_of_text|><|start_header_id|>system<|en...,172
2,What significant invention did NVIDIA create i...,Our invention of the GPU in 1999 defined moder...,NVIDIA invented the GPU in 1999.,<|begin_of_text|><|start_header_id|>system<|en...,74
3,How does NVIDIA's platform strategy contribute...,"NVIDIA has a platform strategy, bringing toget...",NVIDIA's platform strategy brings together har...,<|begin_of_text|><|start_header_id|>system<|en...,98
4,What does NVIDIA's CUDA programming model enable?,With our introduction of the CUDA programming ...,NVIDIA's CUDA programming model opened the par...,<|begin_of_text|><|start_header_id|>system<|en...,84


In [23]:
print(df.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What area did NVIDIA initially focus on before expanding to other computationally intensive fields?

Information:

```
Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>

NVIDIA initially focused on PC graphics.<|eot_id|>


In [25]:
df = df[df.token_count < 512]
df = df.sample(6000)
df.shape

(6000, 5)

In [26]:
train, temp = train_test_split(df, test_size=0.2)
val, test = train_test_split(temp, test_size=0.2)

In [27]:
len(train), len(val), len(test)

(4800, 960, 240)

In [28]:
train.sample(n=4000).to_json("train.json", orient="records", lines=True)
val.sample(n=500).to_json("val.json", orient="records", lines=True)
test.sample(n=100).to_json("test.json", orient="records", lines=True)

In [29]:
dataset = load_dataset(
    "json",
    data_files={"train": "train.json", "validation": "val.json", "test": "test.json"},
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer', 'text', 'token_count'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question', 'context', 'answer', 'text', 'token_count'],
        num_rows: 500
    })
    test: Dataset({
        features: ['question', 'context', 'answer', 'text', 'token_count'],
        num_rows: 100
    })
})

In [31]:
print(dataset["train"][0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the impact of the new $1.25 price point on Dollar Tree’s sales units and profitability?

Information:

```
To date, the increase in the price point has more than offset the decline in the number of units sold.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The increase to a $1.25 price point at Dollar Tree stores has more than offset the decline in the number of units sold, suggesting a positive impact on profitability.<|eot_id|>


In [32]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)

In [33]:
def create_test_prompt(data_row):
    prompt = dedent(
        f"""
    {data_row["question"]}

    Information:

    ```
    {data_row["context"]}
    ```
    """
    )
    messages = [
        {
            "role": "system",
            "content": "Use only the information to answer the question",
        },
        {"role": "user", "content": prompt},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [34]:
row = dataset["test"][0]
prompt = create_test_prompt(row)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Use only the information to answer the question<|eot_id|><|start_header_id|>user<|end_header_id|>

What are the operating segments of the company in Macao?

Information:

```
Our operating segments in Macao consist of The Venetian Macao; The Londoner Macao; The Parisian Macao; The Plaza Macao and Four Seasons Macao; and the Sands Macao.
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [35]:
%%time
outputs = pipe(prompt)
response = f"""
answer:     {row["answer"]}
prediction: {outputs[0]["generated_text"]}
"""
print(response)


answer:     The operating segments in Macao consist of The Venetian Macao, The Londoner Macao, The Parisian Macao, The Plaza Macao and Four Seasons Macao, and the Sands Macao.
prediction: The operating segments of the company in Macao are:

1. The Venetian Macao
2. The Londoner Macao
3. The Parisian Macao
4. The Plaza Macao and Four Seasons Macao
5. Sands Macao

CPU times: user 6.31 s, sys: 404 ms, total: 6.71 s
Wall time: 7.9 s


In [41]:
response_template = "<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

examples = [dataset["train"][0]["text"]]
encodings = [tokenizer(e) for e in examples]

dataloader = DataLoader(encodings, collate_fn=collator, batch_size=1)

In [37]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [38]:
model.print_trainable_parameters()

trainable params: 83,886,080 || all params: 8,114,212,864 || trainable%: 1.0338


In [5]:
OUTPUT_DIR = "experiments"

%load_ext tensorboard
%tensorboard --logdir "experiments/runs"

Reusing TensorBoard on port 6006 (pid 5508), started 1:55:56 ago. (Use '!kill 5508' to kill it.)

<IPython.core.display.Javascript object>

In [42]:
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    dataset_text_field="text",
    max_seq_length=512,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=0.2,
    save_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True,
    save_strategy="steps",
    warmup_ratio=0.1,
    save_total_limit=2,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False,  # No need to add additional separator token
    },
    seed=SEED,
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [43]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.474800,0.454250
200,0.421300,0.444700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_h

Step,Training Loss,Validation Loss
100,0.474800,0.454250
200,0.421300,0.444700
300,0.372800,0.437575
400,0.406200,0.439765
500,0.341600,0.432877


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_h

TrainOutput(global_step=500, training_loss=0.4436292543411255, metrics={'train_runtime': 5990.3696, 'train_samples_per_second': 0.668, 'train_steps_per_second': 0.083, 'total_flos': 2.50236043997184e+16, 'train_loss': 0.4436292543411255, 'epoch': 1.0})

In [44]:
trainer.save_model(NEW_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
model.push_to_hub(NEW_MODEL, use_temp_dir=False, token="")
tokenizer.push_to_hub(NEW_MODEL, use_temp_dir=False, token="")